<a href="https://colab.research.google.com/github/jawaharganesh24189/DLA/blob/main/Copy_of_8E_Adversarial_Dialogue_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adversarial Dialogue Generation System for Custom Dialogue Dataset
## Using GAN Architecture (SeqGAN/LeakGAN)

**Author**: Deep Learning Academy  
**Date**: 2024

This notebook implements a comprehensive adversarial dialogue generation system using Generative Adversarial Networks (GANs) specifically designed for text generation. We apply this to generate dialogue scripts from custom datasets with character-specific dialogue.

### Table of Contents
1. **Background on GANs for Text** - Theory and concepts
2. **Data Preparation** - Load and process Dialogue scripts
3. **Generator Architecture** - LSTM-based sequence generator
4. **Discriminator Architecture** - CNN-based classifier
5. **Policy Gradient** - REINFORCE algorithm
6. **Adversarial Training** - Complete training loop
7. **Autocomplete Tool** - Interactive dialogue generation
8. **Evaluation Metrics** - Quality assessment
9. **Advanced Features** - Conditional generation
10. **Visualizations** - Training dynamics
11. **Model Persistence** - Save/load models
12. **Future Work** - API deployment

## Section 1.0: Background on GANs for Text Generation

### Overview of SeqGAN and LeakGAN

**Traditional Language Models** use maximum likelihood estimation (MLE) to predict the next token. While effective, they suffer from:
- **Exposure bias**: Training vs inference mismatch
- **Loss-evaluation mismatch**: Word-level loss doesn't capture sequence quality
- **Limited diversity**: Tends to generate safe, generic sequences

**Generative Adversarial Networks (GANs)** for text introduce adversarial training to address these issues.

#### Key Concepts:

**1. SeqGAN (Sequence GAN)**:
- Generator: LSTM/Transformer that generates sequences token-by-token
- Discriminator: CNN/LSTM classifier that distinguishes real vs fake sequences
- Uses REINFORCE algorithm (policy gradient) for training discrete sequences
- Monte Carlo rollouts for intermediate reward estimation

**2. LeakGAN (Long Text Generation GAN)**:
- Improves SeqGAN by "leaking" discriminator features to generator
- Manager-Worker hierarchical framework for long-term planning
- Better handling of long sequences and semantic coherence

#### Why Adversarial Training Improves Quality:

1. **Better Reward Signal**: Discriminator provides holistic sequence-level feedback rather than token-level
2. **Increased Diversity**: Generator learns to produce varied outputs to fool discriminator
3. **Reduced Exposure Bias**: Generator is trained on its own outputs during adversarial phase
4. **Quality-Aware Learning**: Discriminator learns what makes text "realistic" and "high-quality"

#### Generator vs Discriminator Roles:

**Generator (G)**:
- **Goal**: Generate realistic dialogue sequences
- **Input**: Start tokens + optional context (character, scene)
- **Output**: Complete token sequence
- **Training**: Policy gradient with discriminator reward
- **Architecture**: LSTM/Transformer with embedding → recurrent layers → softmax output

**Discriminator (D)**:
- **Goal**: Distinguish real (human-written) from fake (generated) dialogue
- **Input**: Complete text sequence
- **Output**: Probability that sequence is real P(real) ∈ [0, 1]
- **Training**: Binary cross-entropy (real vs fake)
- **Architecture**: Embedding → CNN/LSTM → dense layers → sigmoid output

#### Mathematical Framework:

The minimax game:

$$V(D, G) = \mathbb{E}_{x \sim p_{data}}[\log D(x)] + \mathbb{E}_{z \sim p_z}[\log(1 - D(G(z)))]$$

Where:
- $D$ aims to maximize $V(D, G)$ (correctly classify real vs fake)
- $G$ aims to minimize $V(D, G)$ (fool the discriminator)
- At Nash equilibrium: $p_g = p_{data}$ (generator distribution matches real data)

Policy Gradient for Generator:

$$\nabla_\theta J(\theta) = \mathbb{E}_{y_{1:T} \sim G_\theta}[\sum_{t=1}^{T}\nabla_\theta \log G_\theta(y_t|y_{1:t-1}) \cdot Q_{D_\phi}^{G_\theta}(y_{1:t-1}, y_t)]$$

Where $Q$ is the action-value function estimated via Monte Carlo search with discriminator reward.

## Configuration: Dataset Setup

**Before running this notebook, configure your dataset:**

### Dataset Format Requirements

Your dataset should be a text file with dialogue formatted as:
```
[SCENE: Scene Description]
Character1: Dialogue text here
Character2: Response text here

[SCENE: Another Scene]
Character1: More dialogue
```

### Configuration Options

1. **Dataset Path**: Point to your dialogue dataset file
2. **Dataset Folder**: Organize multiple files in a folder
3. **Format**: The parser expects `[SCENE: ...]` and `Character: dialogue` format

**Examples:**
- Movie scripts (formatted as above)
- TV show transcripts
- Play dialogues
- Chat conversations
- Customer service dialogues

**Note**: If you have a different format, modify the `DialogueDataProcessor` class in Section 2.0

In [5]:
# ============================================
# DATASET CONFIGURATION
# ============================================

# Option 1: Single file path
DATA_PATH = '/path/to/your/dataset/dialogues.txt'

# Option 2: Dataset folder with multiple .txt files (NEW!)
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = '/content/drive/MyDrive/DLA_Notebooks_Data_PGPM/Dataset'

# Option 3: Use messy dataset example (for testing flexible format)
USE_MESSY_EXAMPLE = False  # Set to True to test flexible format

# Option 4: Use sample data (original format for testing)
USE_SAMPLE_DATA = False  # Set to False when using your own dataset



# Verify path exists
import os
if os.path.exists(DATA_PATH):
    if os.path.isfile(DATA_PATH):
        print(f"✓ Dataset file found: {DATA_PATH}")
        print(f"  File size: {os.path.getsize(DATA_PATH) / 1024:.2f} KB")
    elif os.path.isdir(DATA_PATH):
        import glob
        txt_files = glob.glob(os.path.join(DATA_PATH, '*.txt'))
        print(f"✓ Dataset folder found: {DATA_PATH}")
        print(f"  Contains {len(txt_files)} .txt file(s)")
        for f in txt_files[:5]:  # Show first 5 files
            print(f"    - {os.path.basename(f)}")
        if len(txt_files) > 5:
            print(f"    ... and {len(txt_files) - 5} more")
else:
    print(f"✗ Dataset path not found: {DATA_PATH}")
    print("  Please update DATA_PATH to point to your dataset file or folder")

# Advanced Options for Flexible Format Parsing
FLEXIBLE_PARSING_OPTIONS = {
    'min_char_occurrence': 2,    # Minimum times a character must appear to be recognized
    'context_as_scene': True,    # Use context lines to enrich scene descriptions
}

print(f"\nParsing options: {FLEXIBLE_PARSING_OPTIONS}")

Mounted at /content/drive
✓ Dataset folder found: /content/drive/MyDrive/DLA_Notebooks_Data_PGPM/Dataset
  Contains 3836 .txt file(s)
    - train-anime-3610.txt
    - train-anime-3621.txt
    - train-anime-3627.txt
    - train-anime-3563.txt
    - train-anime-3632.txt
    ... and 3831 more

Parsing options: {'min_char_occurrence': 2, 'context_as_scene': True}


In [6]:
# Install required packages
!pip install tensorflow==2.13.0 numpy matplotlib seaborn scikit-learn nltk ipywidgets wordcloud -q
print('Packages installed successfully!')

ERROR: Could not find a version that satisfies the requirement tensorflow==2.13.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.13.0
Packages installed successfully!


In [2]:
# Import libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
import pickle
from collections import defaultdict, Counter
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud
import warnings
import os
from pathlib import Path

warnings.filterwarnings('ignore')
nltk.download('punkt', quiet=True)

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Configure GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print(f'GPU available: {physical_devices[0]}')
else:
    print('No GPU found, using CPU')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print(f'TensorFlow version: {tf.__version__}')
print(f'Keras version: {keras.__version__}')

No GPU found, using CPU
TensorFlow version: 2.19.0
Keras version: 3.10.0


## Section 2.0: Data Preparation

In this section, we'll explore two approaches for data preparation:

**Approach 1: DialogueParser Utility** (Optional preprocessing)
- Standalone utility for format conversion and analysis
- Good for preprocessing multiple datasets
- Supports JSONL, CSV, conversational formats

**Approach 2: FlexibleDialogueDataProcessor** (Direct integration)
- Integrated with the training pipeline
- Handles multiple files and flexible formats
- Direct tokenization and sequence creation

You can use both: first preprocess with DialogueParser, then train with FlexibleDialogueDataProcessor.

**Core steps:**
1. **Load** Custom Dialogue Season 1 script from DialogueS1.txt
2. **Parse** characters, dialogue, and scenes
3. **Create** character-specific datasets
4. **Tokenize** text and create sequences
5. **Split** into train/validation sets (85/15)
6. **Augment** data for better generalization

### Data Format:
The script follows this structure:
```
[SCENE: Location]
Character: Dialogue text
```

### Preprocessing Pipeline:
1. Extract scene types (Training, Battle, Strategy, etc.)
2. Extract character names and their dialogues
3. Create context-aware sequences (include previous dialogue for context)
4. Generate sliding windows for sequence learning
5. Build vocabulary and tokenize (max 5000 tokens)
6. Pad sequences to uniform length (50-100 tokens)

### Section 2.1: DialogueParser Utility (Alternative Preprocessing)

Before using the integrated `FlexibleDialogueDataProcessor`, you can optionally preprocess your data using the standalone `DialogueParser` utility. This is useful for:

**Use Cases:**
- Converting between different dialogue formats (JSONL, CSV, conversational)
- Analyzing dataset statistics before training
- Preprocessing multiple dataset files into a single format
- Working with context-response pair formats

**When to Use:**
- **DialogueParser**: Preprocessing, format conversion, batch processing, statistics
- **FlexibleDialogueDataProcessor**: Direct integration with GAN training pipeline

You can use both together: first preprocess with DialogueParser, then load the result with FlexibleDialogueDataProcessor.

In [8]:
# Import the DialogueParser utility
from dialogue_parser import DialogueParser, DatasetStatistics

# Option 1: Use DialogueParser for preprocessing (optional step)
# Uncomment to preprocess your data before training

# Initialize parser
# dialogue_parser = DialogueParser()

# Example 1: Parse a single file
# turns = dialogue_parser.parse_file('sample_dialogues.txt')

# Example 2: Parse entire directory
# all_turns = dialogue_parser.parse_directory('messy_dataset/')

# Example 3: Calculate statistics
# stats = DatasetStatistics.calculate_stats(all_turns)
# print('Dataset Statistics:')
# for key, value in stats.items():
#     print(f'  {key}: {value:.2f}' if isinstance(value, float) else f'  {key}: {value}')

# Example 4: Convert to training format and save
# jsonl_output = dialogue_parser.to_training_format(all_turns, 'jsonl')
# with open('preprocessed_data.jsonl', 'w') as f:
#     f.write(jsonl_output)

# Example 5: Convert to CSV for analysis
# csv_output = dialogue_parser.to_training_format(all_turns, 'csv')
# with open('dialogues_analysis.csv', 'w') as f:
#     f.write(csv_output)

print('DialogueParser utility available for optional preprocessing')
print('Uncomment examples above to use it before the main training pipeline')
print('\nFor more details, see DIALOGUE_PARSER_GUIDE.md')

DialogueParser utility available for optional preprocessing
Uncomment examples above to use it before the main training pipeline

For more details, see DIALOGUE_PARSER_GUIDE.md


**Integration Workflow:**

```python
# Workflow 1: Direct training (default)
DATA_PATH = 'sample_dialogues.txt'  # or folder path
# → Use FlexibleDialogueDataProcessor below

# Workflow 2: Preprocess then train
# Step 1: Preprocess with DialogueParser
dialogue_parser = DialogueParser()
turns = dialogue_parser.parse_directory('raw_dataset/')
output = dialogue_parser.to_training_format(turns, 'jsonl')
with open('clean_data.jsonl', 'w') as f:
    f.write(output)

# Step 2: Set DATA_PATH to preprocessed file
DATA_PATH = 'clean_data.jsonl'
# → Continue with FlexibleDialogueDataProcessor
```

---

In [ ]:
dialogue_parser = DialogueParser()
turns = dialogue_parser.parse_directory('/content/drive/MyDrive/DLA_Notebooks_Data_PGPM/Dataset/')
output = dialogue_parser.to_training_format(turns, 'jsonl')
with open('clean_data.jsonl', 'w') as f:
    f.write(output)

# Step 2: Set DATA_PATH to preprocessed file
DATA_PATH = 'clean_data.jsonl'


import os
import glob
import re
from collections import defaultdict

class FlexibleDialogueDataProcessor:
    """
    Enhanced data processor that handles:
    1. Multiple .txt files in a folder
    2. Flexible formats (context lines, character: dialogue, etc.)
    3. Smart detection of character names vs context
    """

    def __init__(self, file_path_or_folder, seq_length=50, max_vocab_size=5000,
                 min_char_occurrence=2, context_as_scene=True):
        """
        Args:
            file_path_or_folder: Path to a single file OR a folder containing .txt files
            seq_length: Maximum sequence length
            max_vocab_size: Maximum vocabulary size
            min_char_occurrence: Minimum times a character name must appear to be recognized
            context_as_scene: Whether to treat context lines as scene descriptions
        """
        self.path = file_path_or_folder
        self.seq_length = seq_length
        self.max_vocab_size = max_vocab_size
        self.min_char_occurrence = min_char_occurrence
        self.context_as_scene = context_as_scene

        self.dialogues = []
        self.characters = defaultdict(list)
        self.scenes = defaultdict(list)
        self.tokenizer = None
        self.vocab_size = 0

    def _get_file_list(self):
        """Get list of files to process"""
        if os.path.isfile(self.path):
            return [self.path]
        elif os.path.isdir(self.path):
            # Get all .txt files in the folder
            txt_files = glob.glob(os.path.join(self.path, '*.txt'))
            if not txt_files:
                print(f"Warning: No .txt files found in {self.path}")
            return sorted(txt_files)
        else:
            raise ValueError(f"Path not found: {self.path}")

    def _is_likely_character_line(self, line):
        """
        Detect if a line is likely "Character: dialogue" format
        Returns: (is_character_line, character_name, dialogue_text)
        """
        # Must have a colon
        if ':' not in line:
            return False, None, None

        # Split on first colon
        parts = line.split(':', 1)
        if len(parts) != 2:
            return False, None, None

        potential_char = parts[0].strip()
        dialogue = parts[1].strip()

        # Character name should be short (1-3 words) and capitalized
        words = potential_char.split()
        if len(words) > 3:
            return False, None, None

        # Check if it looks like a character name (starts with capital, no weird punctuation)
        if not potential_char or not potential_char[0].isupper():
            return False, None, None

        # Check for common non-character patterns
        lowercase_potential = potential_char.lower()
        if any(keyword in lowercase_potential for keyword in ['note', 'context', 'scene', 'time', 'location']):
            return False, None, None

        # Dialogue should exist and not be super long (context lines tend to be longer)
        if not dialogue or len(dialogue) < 5:
            return False, None, None

        return True, potential_char, dialogue

    def _is_scene_marker(self, line):
        """Check if line is a scene marker like [SCENE: Description]"""
        return line.startswith('[SCENE:') and line.endswith(']')

    def _extract_scene(self, line):
        """Extract scene description from marker"""
        if self._is_scene_marker(line):
            return line[7:-1].strip()
        return None

    def load_and_parse(self, verbose=True):
        """Load and parse files with flexible format support"""
        files = self._get_file_list()

        if verbose:
            print(f"Processing {len(files)} file(s)...")

        # First pass: collect all potential character names
        potential_characters = defaultdict(int)
        all_lines = []

        for file_path in files:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read()

            lines = content.split('\n')
            for line in lines:
                line = line.strip()
                if not line:
                    continue

                all_lines.append((file_path, line))

                is_char, char_name, _ = self._is_likely_character_line(line)
                if is_char:
                    potential_characters[char_name] += 1

        # Filter characters by minimum occurrence
        valid_characters = {
            char for char, count in potential_characters.items()
            if count >= self.min_char_occurrence
        }

        if verbose:
            print(f"Detected {len(valid_characters)} characters: {sorted(valid_characters)}")

        # Second pass: parse dialogues with validated characters
        current_scene = 'Unknown'
        context_buffer = []

        for file_path, line in all_lines:
            # Check for scene marker
            scene = self._extract_scene(line)
            if scene:
                current_scene = scene
                context_buffer = []
                continue

            # Check for character dialogue
            is_char, char_name, dialogue = self._is_likely_character_line(line)

            if is_char and char_name in valid_characters:
                # This is valid character dialogue
                # Use context buffer as additional scene info if enabled
                if self.context_as_scene and context_buffer:
                    scene_with_context = f"{current_scene} ({' '.join(context_buffer[:2])})"
                else:
                    scene_with_context = current_scene

                entry = {
                    'character': char_name,
                    'dialogue': dialogue,
                    'scene': scene_with_context,
                    'source_file': os.path.basename(file_path)
                }

                self.dialogues.append(entry)
                self.characters[char_name].append(dialogue)
                self.scenes[scene_with_context].append(dialogue)

                # Clear context buffer after use
                context_buffer = []
            else:
                # This is a context line - store it
                if len(context_buffer) < 5:  # Limit context buffer size
                    context_buffer.append(line[:50])  # Truncate long lines

        if verbose:
            print(f'\nLoaded {len(self.dialogues)} dialogue entries')
            print(f'Characters ({len(self.characters)}): {list(self.characters.keys())}')
            print(f'Unique scenes: {len(self.scenes)}')

            # Character statistics
            print('\nDialogue distribution by character:')
            for char, dialogues in sorted(self.characters.items(),
                                         key=lambda x: len(x[1]), reverse=True)[:10]:
                print(f'  {char}: {len(dialogues)} lines')

        return self.dialogues

    def create_tokenizer(self):
        """Create and fit tokenizer on dialogue data"""
        all_dialogues = [d['dialogue'] for d in self.dialogues]

        self.tokenizer = Tokenizer(
            num_words=self.max_vocab_size,
            filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
            lower=True,
            oov_token='<UNK>'
        )

        self.tokenizer.fit_on_texts(all_dialogues)
        self.vocab_size = min(len(self.tokenizer.word_index) + 1, self.max_vocab_size)

        print(f'\nVocabulary size: {self.vocab_size}')
        print(f'Most common words: {list(self.tokenizer.word_index.items())[:20]}')

        return self.tokenizer

    def create_sequences(self, use_context=True):
        """Create training sequences with sliding window"""
        sequences = []
        labels = []  # For discriminator (1 = real)
        metadata = []  # Character and scene info

        for i, entry in enumerate(self.dialogues):
            # Get context (previous dialogue if available)
            context = ''
            if use_context and i > 0:
                context = self.dialogues[i-1]['dialogue'] + ' '

            full_text = context + entry['dialogue']
            token_list = self.tokenizer.texts_to_sequences([full_text])[0]

            # Create sliding windows
            for j in range(1, len(token_list)):
                n_gram_sequence = token_list[:j+1]
                if len(n_gram_sequence) <= self.seq_length:
                    sequences.append(n_gram_sequence)
                    labels.append(1)  # Real data
                    metadata.append({
                        'character': entry['character'],
                        'scene': entry['scene']
                    })

        # Pad sequences
        padded_sequences = pad_sequences(
            sequences,
            maxlen=self.seq_length,
            padding='pre'
        )

        print(f'\nCreated {len(padded_sequences)} sequences')
        print(f'Sequence shape: {padded_sequences.shape}')

        return padded_sequences, np.array(labels), metadata

    def augment_data(self, sequences, metadata, augment_factor=2):
        """Apply data augmentation techniques"""
        print(f'\nApplying data augmentation (factor={augment_factor})...')
        augmented_seqs = [sequences]
        augmented_meta = [metadata]

        for aug_idx in range(augment_factor - 1):
            # Random token dropout (5%)
            aug_seq = sequences.copy()
            mask = np.random.random(aug_seq.shape) > 0.05
            aug_seq = aug_seq * mask.astype(int)

            augmented_seqs.append(aug_seq)
            augmented_meta.append(metadata.copy())

        combined_seqs = np.vstack(augmented_seqs)
        combined_meta = []
        for meta_list in augmented_meta:
            combined_meta.extend(meta_list)

        print(f'Augmented from {sequences.shape[0]} to {combined_seqs.shape[0]} sequences')
        return combined_seqs, combined_meta

    def get_character_encoding(self):
        """Create character to index mapping"""
        unique_chars = sorted(list(self.characters.keys()))
        char_to_idx = {char: idx for idx, char in enumerate(unique_chars)}
        idx_to_char = {idx: char for char, idx in char_to_idx.items()}
        return char_to_idx, idx_to_char, len(unique_chars)

    def get_scene_encoding(self):
        """Create scene to index mapping"""
        unique_scenes = sorted(list(self.scenes.keys()))
        scene_to_idx = {scene: idx for idx, scene in enumerate(unique_scenes)}
        idx_to_scene = {idx: scene for scene, idx in scene_to_idx.items()}
        return scene_to_idx, idx_to_scene, len(unique_scenes)

# Keep old class for backward compatibility
DialogueDataProcessor = FlexibleDialogueDataProcessor

# Initialize data processor
print('='*80)
print('INITIALIZING FLEXIBLE DATA PROCESSOR')
print('='*80)
print(f'Dataset path: {DATA_PATH}')
print(f'Can handle: single file OR folder with multiple .txt files')
print(f'Format support: flexible (context lines + character: dialogue)')
print('='*80)

data_processor = FlexibleDialogueDataProcessor(
    file_path_or_folder=DATA_PATH,  # Can be file or folder
    seq_length=50,
    max_vocab_size=5000,
    min_char_occurrence=2,  # Character must appear at least 2 times
    context_as_scene=True   # Use context lines to enrich scene info
)

# Load and parse data
dialogues = data_processor.load_and_parse()


In [ ]:
# Create tokenizer
tokenizer = data_processor.create_tokenizer()

# Create sequences
sequences, labels, metadata = data_processor.create_sequences(use_context=True)

# Get character and scene encodings
char_to_idx, idx_to_char, num_characters = data_processor.get_character_encoding()
scene_to_idx, idx_to_scene, num_scenes = data_processor.get_scene_encoding()

print(f'\nCharacter mapping ({num_characters} characters):')
for char, idx in sorted(char_to_idx.items(), key=lambda x: x[1]):
    print(f'  {idx}: {char}')

print(f'\nScene mapping ({num_scenes} scenes):')
for scene, idx in sorted(scene_to_idx.items(), key=lambda x: x[1]):
    print(f'  {idx}: {scene}')

In [ ]:
# Create character and scene condition arrays
char_conditions = np.array([char_to_idx[m['character']] for m in metadata])
scene_conditions = np.array([scene_to_idx[m['scene']] for m in metadata])

# Apply data augmentation
sequences_aug, metadata_aug = data_processor.augment_data(
    sequences, metadata, augment_factor=2
)

# Update condition arrays for augmented data
char_conditions_aug = np.array([char_to_idx[m['character']] for m in metadata_aug])
scene_conditions_aug = np.array([scene_to_idx[m['scene']] for m in metadata_aug])

# Train/validation split (85/15)
train_size = int(0.85 * len(sequences_aug))
indices = np.arange(len(sequences_aug))
np.random.shuffle(indices)

train_indices = indices[:train_size]
val_indices = indices[train_size:]

X_train = sequences_aug[train_indices]
X_val = sequences_aug[val_indices]

char_train = char_conditions_aug[train_indices]
char_val = char_conditions_aug[val_indices]

scene_train = scene_conditions_aug[train_indices]
scene_val = scene_conditions_aug[val_indices]

print(f'\n{'='*80}')
print('DATASET SUMMARY')
print('='*80)
print(f'Training set: {X_train.shape[0]} sequences')
print(f'Validation set: {X_val.shape[0]} sequences')
print(f'Sequence length: {X_train.shape[1]}')
print(f'Vocabulary size: {data_processor.vocab_size}')
print(f'\nSample sequence (tokens): {X_train[0]}')
decoded = ' '.join([tokenizer.index_word.get(idx, '<PAD>') for idx in X_train[0] if idx > 0])
print(f'Decoded: {decoded}')
print('='*80)

## Section 3.0: Generator Architecture

The Generator is responsible for creating realistic dialogue sequences. Our architecture combines:

### Architecture Components:

**1. Embedding Layer** (256 dimensions)
- Converts token indices to dense vector representations
- Learnable embeddings capture semantic relationships
- Shared vocabulary space with discriminator

**2. LSTM Layers** (2-3 layers, 512 units each)
- Captures long-range sequential dependencies
- Stacked for hierarchical feature learning
- Dropout (0.3) for regularization and preventing overfitting
- Return sequences for multi-layer stacking

**3. Conditional Inputs** (Optional)
- **Character embedding**: Enables character-specific dialogue generation
- **Scene embedding**: Generates scene-appropriate dialogue
- Concatenated with token embeddings at each timestep
- Provides additional context for generation

**4. Output Layer**
- Dense layer with vocabulary size output
- Softmax activation produces probability distribution
- Temperature sampling controls randomness/diversity

### Generation Strategies:

**Temperature Sampling**:
- Temperature T controls randomness
- T < 1: More conservative, focused on high-probability tokens
- T = 1: Standard sampling from distribution
- T > 1: More diverse, exploratory generation

**Decoding Methods**:
- **Greedy**: Always select highest probability token
- **Sampling**: Sample from probability distribution
- **Beam Search**: Keep top-k sequences (future enhancement)

### Training Phases:
- **Pre-training**: Teacher forcing with MLE
- **Adversarial**: Self-sampling with policy gradient

In [ ]:
class Generator(Model):
    """LSTM-based sequence generator with conditional inputs"""

    def __init__(self, vocab_size, embedding_dim=256, lstm_units=512,
                 num_lstm_layers=2, dropout=0.3, num_characters=0, num_scenes=0):
        super(Generator, self).__init__()

        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.lstm_units = lstm_units
        self.num_lstm_layers = num_lstm_layers

        # Token embedding
        self.token_embedding = layers.Embedding(
            vocab_size, embedding_dim, mask_zero=True, name='token_embedding'
        )

        # Conditional embeddings
        self.use_char_condition = num_characters > 0
        self.use_scene_condition = num_scenes > 0

        if self.use_char_condition:
            self.char_embedding = layers.Embedding(
                num_characters, 32, name='char_embedding'
            )

        if self.use_scene_condition:
            self.scene_embedding = layers.Embedding(
                num_scenes, 32, name='scene_embedding'
            )

        # Calculate LSTM input dimension
        lstm_input_dim = embedding_dim
        if self.use_char_condition:
            lstm_input_dim += 32
        if self.use_scene_condition:
            lstm_input_dim += 32

        # LSTM layers
        self.lstm_layers = []
        for i in range(num_lstm_layers):
            self.lstm_layers.append(
                layers.LSTM(
                    lstm_units,
                    return_sequences=True,
                    dropout=dropout,
                    recurrent_dropout=0.0,  # Can cause issues with CuDNN
                    name=f'lstm_{i}'
                )
            )

        # Output layer
        self.output_layer = layers.Dense(vocab_size, name='output')

    def call(self, inputs, training=False, char_condition=None, scene_condition=None):
        # Token embedding
        x = self.token_embedding(inputs)

        # Add conditional embeddings if provided
        if char_condition is not None and self.use_char_condition:
            char_emb = self.char_embedding(char_condition)
            # Expand and tile to match sequence length
            char_emb = tf.expand_dims(char_emb, 1)
            char_emb = tf.tile(char_emb, [1, tf.shape(x)[1], 1])
            x = tf.concat([x, char_emb], axis=-1)

        if scene_condition is not None and self.use_scene_condition:
            scene_emb = self.scene_embedding(scene_condition)
            scene_emb = tf.expand_dims(scene_emb, 1)
            scene_emb = tf.tile(scene_emb, [1, tf.shape(x)[1], 1])
            x = tf.concat([x, scene_emb], axis=-1)

        # LSTM layers
        for lstm in self.lstm_layers:
            x = lstm(x, training=training)

        # Output logits
        logits = self.output_layer(x)
        return logits

    def generate_sequence(self, start_tokens, max_length=50, temperature=1.0,
                         char_condition=None, scene_condition=None, method='sampling'):
        """
        Generate sequence using different decoding strategies

        Args:
            start_tokens: Initial tokens [batch_size, start_len]
            max_length: Maximum sequence length
            temperature: Sampling temperature (higher = more random)
            char_condition: Character condition index
            scene_condition: Scene condition index
            method: 'greedy' or 'sampling'

        Returns:
            Generated sequences [batch_size, max_length]
        """
        current_seq = start_tokens

        for step in range(max_length - tf.shape(start_tokens)[1]):
            # Get predictions
            logits = self(current_seq, training=False,
                         char_condition=char_condition,
                         scene_condition=scene_condition)

            # Get last token predictions
            next_token_logits = logits[:, -1, :] / temperature

            if method == 'greedy':
                next_token = tf.argmax(next_token_logits, axis=-1, output_type=tf.int32)
            elif method == 'sampling':
                next_token = tf.random.categorical(next_token_logits, 1)
                next_token = tf.squeeze(next_token, axis=-1)
            else:
                next_token = tf.argmax(next_token_logits, axis=-1, output_type=tf.int32)

            # Append to sequence
            next_token = tf.expand_dims(next_token, -1)
            current_seq = tf.concat([current_seq, next_token], axis=1)

            # Early stopping if all sequences generated EOS (if implemented)
            # if tf.reduce_all(next_token == eos_token_id):
            #     break

        return current_seq

# Create generator
print('='*80)
print('BUILDING GENERATOR')
print('='*80)

generator = Generator(
    vocab_size=data_processor.vocab_size,
    embedding_dim=256,
    lstm_units=512,
    num_lstm_layers=2,
    dropout=0.3,
    num_characters=num_characters,
    num_scenes=num_scenes
)

# Build model
sample_input = tf.random.uniform((2, 10), maxval=data_processor.vocab_size, dtype=tf.int32)
sample_char = tf.constant([0, 1])
sample_scene = tf.constant([0, 1])
_ = generator(sample_input, char_condition=sample_char, scene_condition=sample_scene)

print('\nGenerator Architecture:')
generator.summary()
print(f'\nTotal parameters: {generator.count_params():,}')
print('='*80)

In [ ]:
# Test generator generation
print('Testing generator sequence generation...')
test_start = tf.constant([[1, 2, 3]])  # Sample start tokens
test_char = tf.constant([0])  # First character
test_scene = tf.constant([0])  # First scene

generated = generator.generate_sequence(
    test_start,
    max_length=20,
    temperature=1.0,
    char_condition=test_char,
    scene_condition=test_scene,
    method='sampling'
)

print(f'Generated sequence shape: {generated.shape}')
print(f'Generated tokens: {generated[0].numpy()}')
print('\nGenerator test successful!')

## Section 3.1: Discriminator Architecture

The Discriminator classifies sequences as real (human-written) or fake (generated).

### CNN-Based Architecture:

We implement a CNN-based discriminator for efficient local pattern recognition:

**1. Embedding Layer**
- Separate embedding from generator (not shared)
- 256-dimensional vectors
- Learns to recognize real vs fake patterns

**2. Multiple 1D Convolutional Layers**
- Different kernel sizes (3, 4, 5) capture n-gram features
- Kernels learn local patterns (2-grams, 3-grams, etc.)
- Parallel convolutions for multi-scale feature extraction
- ReLU activation for non-linearity

**3. Global Max Pooling**
- Extracts most significant feature from each filter
- Position-invariant feature detection
- Reduces dimensionality

**4. Dense Classification Head**
- Multiple dense layers with ReLU
- Dropout (0.3) for regularization
- Final sigmoid outputs P(real) ∈ [0, 1]

### Training Techniques:

**Label Smoothing**:
- Real labels: 0.9 instead of 1.0
- Fake labels: 0.1 instead of 0.0
- Prevents discriminator overconfidence
- Improves gradient flow to generator

**Gradient Penalty** (Optional):
- Encourages 1-Lipschitz continuity
- Stabilizes training
- Alternative to gradient clipping

**One-sided Label Smoothing**:
- Only smooth real labels, not fake
- Prevents mode collapse
- Maintains discriminator gradient quality

In [ ]:
class Discriminator(Model):
    """CNN-based discriminator for sequence classification"""

    def __init__(self, vocab_size, embedding_dim=256, filters=64,
                 kernel_sizes=[3, 4, 5], dropout=0.3):
        super(Discriminator, self).__init__()

        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.filters = filters
        self.kernel_sizes = kernel_sizes

        # Embedding layer (separate from generator)
        self.embedding = layers.Embedding(
            vocab_size, embedding_dim, mask_zero=False, name='d_embedding'
        )

        # Multiple CNN layers with different kernel sizes
        self.conv_layers = []
        self.pool_layers = []

        for kernel_size in kernel_sizes:
            conv = layers.Conv1D(
                filters=filters,
                kernel_size=kernel_size,
                activation='relu',
                padding='same',
                name=f'conv_{kernel_size}'
            )
            pool = layers.GlobalMaxPooling1D(name=f'pool_{kernel_size}')
            self.conv_layers.append(conv)
            self.pool_layers.append(pool)

        # Dense layers
        self.dense1 = layers.Dense(128, activation='relu', name='dense1')
        self.dropout1 = layers.Dropout(dropout, name='dropout1')
        self.dense2 = layers.Dense(64, activation='relu', name='dense2')
        self.dropout2 = layers.Dropout(dropout, name='dropout2')

        # Output layer (sigmoid for probability)
        self.output_layer = layers.Dense(1, activation='sigmoid', name='output')

    def call(self, inputs, training=False):
        # Embedding
        x = self.embedding(inputs)

        # Apply multiple CNNs and pool, then concatenate
        conv_outputs = []
        for conv, pool in zip(self.conv_layers, self.pool_layers):
            conv_out = conv(x)
            pooled = pool(conv_out)
            conv_outputs.append(pooled)

        # Concatenate all CNN outputs
        x = tf.concat(conv_outputs, axis=-1)

        # Dense layers with dropout
        x = self.dense1(x)
        x = self.dropout1(x, training=training)
        x = self.dense2(x)
        x = self.dropout2(x, training=training)

        # Output probability
        output = self.output_layer(x)
        return output

    def get_features(self, inputs):
        """Extract feature representations for visualization (t-SNE)"""
        x = self.embedding(inputs)
        conv_outputs = []
        for conv, pool in zip(self.conv_layers, self.pool_layers):
            conv_out = conv(x)
            pooled = pool(conv_out)
            conv_outputs.append(pooled)
        features = tf.concat(conv_outputs, axis=-1)
        return features

# Create discriminator
print('='*80)
print('BUILDING DISCRIMINATOR')
print('='*80)

discriminator = Discriminator(
    vocab_size=data_processor.vocab_size,
    embedding_dim=256,
    filters=64,
    kernel_sizes=[3, 4, 5],
    dropout=0.3
)

# Build model
sample_input = tf.random.uniform((2, 50), maxval=data_processor.vocab_size, dtype=tf.int32)
_ = discriminator(sample_input)

print('\nDiscriminator Architecture:')
discriminator.summary()
print(f'\nTotal parameters: {discriminator.count_params():,}')
print('='*80)

In [ ]:
# Test discriminator
print('Testing discriminator...')
test_seq = tf.random.uniform((4, 50), maxval=data_processor.vocab_size, dtype=tf.int32)
predictions = discriminator(test_seq, training=False)

print(f'Input shape: {test_seq.shape}')
print(f'Output shape: {predictions.shape}')
print(f'Predictions (P(real)): {predictions.numpy().flatten()}')
print('\nDiscriminator test successful!')

## Section 3.2: Policy Gradient Implementation

Text generation is discrete, so we can't backpropagate through sampling. We use **REINFORCE** (policy gradient).

### REINFORCE Algorithm:

The policy gradient theorem:

$$\nabla_\theta J(\theta) = \mathbb{E}_{y_{1:T} \sim G_\theta}\left[\sum_{t=1}^{T}\nabla_\theta \log G_\theta(y_t|y_{1:t-1}) \cdot Q(y_{1:t})\right]$$

Where:
- $\theta$: Generator parameters
- $G_\theta$: Generator policy
- $y_{1:T}$: Generated sequence
- $Q(y_{1:t})$: Action-value (expected reward)

### Reward Function:

**Complete Sequences**:
$$R(y_{1:T}) = D_\phi(y_{1:T})$$

Direct discriminator output.

**Partial Sequences** (Monte Carlo Rollouts):
$$Q(y_{1:t}) = \frac{1}{N}\sum_{i=1}^{N} D_\phi(y_{1:t}, y_{t+1:T}^{(i)})$$

Where $y_{t+1:T}^{(i)}$ are sampled completions.

### Monte Carlo Rollouts:

For incomplete sequences:
1. Take partial sequence $y_{1:t}$
2. Sample N completions using generator
3. Evaluate each completion with discriminator
4. Average discriminator scores
5. Use as reward estimate

### Baseline for Variance Reduction:

Advantage function:
$$A(y_{1:t}) = Q(y_{1:t}) - b$$

Where $b$ is baseline (e.g., moving average of rewards).

**Benefits**:
- Reduces gradient variance
- Faster convergence
- More stable training

### Implementation Details:

1. **Sample sequences** from generator
2. **Compute log probabilities** for each token
3. **Get rewards** from discriminator
4. **Calculate advantages** (reward - baseline)
5. **Compute policy gradient** loss
6. **Update generator** parameters

In [ ]:
class PolicyGradient:
    """REINFORCE algorithm for generator training"""

    def __init__(self, generator, discriminator, rollout_num=4):
        self.generator = generator
        self.discriminator = discriminator
        self.rollout_num = rollout_num
        self.reward_baseline = 0.5  # Moving average baseline
        self.baseline_momentum = 0.9

    def get_reward(self, sequences, rollout=False, num_rollouts=4):
        """
        Get reward from discriminator

        Args:
            sequences: Generated sequences [batch_size, seq_len]
            rollout: Whether to use Monte Carlo rollouts
            num_rollouts: Number of rollouts for incomplete sequences

        Returns:
            rewards: Reward for each sequence [batch_size]
        """
        # Direct discriminator evaluation
        rewards = self.discriminator(sequences, training=False)
        rewards = tf.squeeze(rewards, axis=-1)

        # Update baseline
        current_mean = tf.reduce_mean(rewards)
        self.reward_baseline = (
            self.baseline_momentum * self.reward_baseline +
            (1 - self.baseline_momentum) * current_mean
        )

        return rewards

    def compute_pg_loss(self, sequences, char_condition=None, scene_condition=None):
        """
        Compute policy gradient loss

        Args:
            sequences: Input sequences [batch_size, seq_len]
            char_condition: Character conditions
            scene_condition: Scene conditions

        Returns:
            loss: Policy gradient loss
            rewards: Average reward
        """
        # Get generator predictions (logits)
        logits = self.generator(sequences[:, :-1], training=True,
                               char_condition=char_condition,
                               scene_condition=scene_condition)

        # Compute log probabilities
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Get log probabilities of selected tokens
        # Create indices for gather_nd
        batch_size = tf.shape(sequences)[0]
        seq_len = tf.shape(sequences)[1] - 1

        batch_indices = tf.tile(
            tf.expand_dims(tf.range(batch_size), 1),
            [1, seq_len]
        )
        time_indices = tf.tile(
            tf.expand_dims(tf.range(seq_len), 0),
            [batch_size, 1]
        )
        token_indices = sequences[:, 1:]

        indices = tf.stack([
            batch_indices,
            time_indices,
            token_indices
        ], axis=-1)

        selected_log_probs = tf.gather_nd(log_probs, indices)

        # Get rewards from discriminator
        rewards = self.get_reward(sequences)

        # Compute advantages (reward - baseline)
        advantages = rewards - self.reward_baseline

        # Policy gradient loss
        # Negative because we want to maximize reward
        pg_loss = -tf.reduce_mean(
            tf.reduce_sum(selected_log_probs, axis=1) * advantages
        )

        return pg_loss, tf.reduce_mean(rewards)

# Create policy gradient trainer
print('='*80)
print('INITIALIZING POLICY GRADIENT TRAINER')
print('='*80)

pg_trainer = PolicyGradient(
    generator=generator,
    discriminator=discriminator,
    rollout_num=4
)

print(f'Rollout samples: {pg_trainer.rollout_num}')
print(f'Initial baseline: {pg_trainer.reward_baseline}')
print(f'Baseline momentum: {pg_trainer.baseline_momentum}')
print('='*80)

## Section 4.0: Adversarial Training Loop

Complete GAN training consists of two phases:

### Phase 1: Pre-training (5-10 epochs each)

**Generator Pre-training**:
- Standard MLE (Maximum Likelihood Estimation)
- Teacher forcing: Use ground truth previous tokens
- Cross-entropy loss
- Prepares generator for adversarial training

**Discriminator Pre-training**:
- Train on real data (label = 1) vs random/pre-trained generator samples (label = 0)
- Binary cross-entropy loss
- Label smoothing: real = 0.9, fake = 0.1
- Provides initial signal for generator

### Phase 2: Adversarial Training (20-50 epochs)

**Training Loop**:
```
for epoch in epochs:
    for batch in data:
        # 1. Train Discriminator (1-2 steps)
        fake_samples = generator.generate()
        d_loss_real = discriminator.train(real_data, label=1)
        d_loss_fake = discriminator.train(fake_samples, label=0)
        
        # 2. Train Generator (1 step)
        fake_samples = generator.generate()
        rewards = discriminator(fake_samples)
        g_loss = policy_gradient(fake_samples, rewards)
```

### Training Techniques:

**1. Label Smoothing**:
- Real labels: 0.9 (not 1.0)
- Fake labels: 0.1 (not 0.0)
- Prevents overconfidence

**2. Gradient Clipping**:
- Clip gradients to [-1, 1]
- Prevents exploding gradients
- Stabilizes training

**3. Learning Rate Scheduling**:
- Start: 1e-4
- Decay: Exponential or step decay
- Separate rates for G and D

**4. Batch Size**:
- 32-128 samples per batch
- Larger batches → more stable gradients
- Smaller batches → faster iterations

**5. Training Ratio**:
- D updates: 1-2 per G update
- Keeps D slightly ahead but not too strong

### Monitoring:

**Metrics to Track**:
- Generator loss (policy gradient)
- Discriminator loss (binary cross-entropy)
- Discriminator accuracy (real vs fake)
- Average reward from discriminator
- Sample quality (manual inspection)
- BLEU score on validation set

**Early Stopping**:
- Monitor validation BLEU score
- Stop if no improvement for N epochs
- Save best model based on metrics

### Handling Issues:

**Mode Collapse**:
- Generator produces limited variety
- Solution: Increase diversity penalty, minibatch discrimination

**Training Instability**:
- Losses oscillate wildly
- Solution: Lower learning rate, stronger regularization

**Discriminator Too Strong**:
- Generator gets no useful gradient
- Solution: Reduce D training steps, increase D dropout

In [ ]:
# HyperparametersBATCH_SIZE = 64SEQ_LENGTH = 50PRETRAIN_EPOCHS_G = 5PRETRAIN_EPOCHS_D = 3ADVERSARIAL_EPOCHS = 20D_STEPS = 1  # Train discriminator every N batches (not every batch)G_STEPS = 1LEARNING_RATE_G = 1e-4LEARNING_RATE_D = 1e-4LABEL_SMOOTHING = 0.1GRADIENT_CLIP = 1.0# Temperature annealing for Gumbel softmaxTEMP_START = 2.0  # High temperature = more explorationTEMP_END = 0.5    # Low temperature = more exploitationTEMP_DECAY = 0.95  # Decay rate per epoch# Performance optimizationsD_TRAIN_INTERVAL = 1  # Train D every N batches (1 = every batch)ACCURACY_METRIC_ON_GPU = True  # Keep accuracy on GPUprint('='*80)print('TRAINING HYPERPARAMETERS')print('='*80)print(f'Batch size: {BATCH_SIZE}')print(f'Sequence length: {SEQ_LENGTH}')print(f'Generator pre-train epochs: {PRETRAIN_EPOCHS_G}')print(f'Discriminator pre-train epochs: {PRETRAIN_EPOCHS_D}')print(f'Adversarial epochs: {ADVERSARIAL_EPOCHS}')print(f'D steps per iteration: {D_STEPS}')print(f'G steps per iteration: {G_STEPS}')print(f'Learning rate (G): {LEARNING_RATE_G}')print(f'Learning rate (D): {LEARNING_RATE_D}')print(f'Label smoothing: {LABEL_SMOOTHING}')print(f'Gradient clipping: {GRADIENT_CLIP}')print(f'Temperature: {TEMP_START} → {TEMP_END} (decay: {TEMP_DECAY})')print(f'D train interval: {D_TRAIN_INTERVAL}')print('='*80)

In [ ]:
# Create optimizers
optimizer_g = Adam(learning_rate=LEARNING_RATE_G, clipnorm=GRADIENT_CLIP)
optimizer_d = Adam(learning_rate=LEARNING_RATE_D, clipnorm=GRADIENT_CLIP)

# Loss functions
bce_loss = keras.losses.BinaryCrossentropy(from_logits=False)
sparse_ce_loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Training history
history = {
    'g_loss': [],
    'd_loss': [],
    'd_loss_real': [],
    'd_loss_fake': [],
    'd_acc': [],
    'g_reward': [],
    'epoch': []
}

print('Optimizers and loss functions initialized')

In [ ]:
# Pre-training functions

@tf.function
def pretrain_generator_step(sequences, char_cond, scene_cond):
    """Pre-train generator with MLE (teacher forcing)"""
    with tf.GradientTape() as tape:
        # Predict next tokens
        logits = generator(sequences[:, :-1], training=True,
                          char_condition=char_cond,
                          scene_condition=scene_cond)

        # MLE loss (cross-entropy)
        loss = sparse_ce_loss(sequences[:, 1:], logits)

    # Update generator
    gradients = tape.gradient(loss, generator.trainable_variables)
    optimizer_g.apply_gradients(zip(gradients, generator.trainable_variables))

    return loss

@tf.function
def pretrain_discriminator_step(real_sequences, fake_sequences):
    """Pre-train discriminator on real vs fake"""
    with tf.GradientTape() as tape:
        # Predictions
        real_pred = discriminator(real_sequences, training=True)
        fake_pred = discriminator(fake_sequences, training=True)

        # Labels with smoothing
        real_labels = tf.ones_like(real_pred) * (1 - LABEL_SMOOTHING)
        fake_labels = tf.ones_like(fake_pred) * LABEL_SMOOTHING

        # Losses
        loss_real = bce_loss(real_labels, real_pred)
        loss_fake = bce_loss(fake_labels, fake_pred)
        loss = loss_real + loss_fake

    # Update discriminator
    gradients = tape.gradient(loss, discriminator.trainable_variables)
    optimizer_d.apply_gradients(zip(gradients, discriminator.trainable_variables))

    # Accuracy
    acc = (tf.reduce_mean(tf.cast(real_pred > 0.5, tf.float32)) +
           tf.reduce_mean(tf.cast(fake_pred < 0.5, tf.float32))) / 2

    return loss, loss_real, loss_fake, acc

print('Pre-training step functions defined')

In [ ]:
# Adversarial training functions@tf.functiondef train_discriminator_adversarial(real_sequences, fake_sequences):    """Train discriminator during adversarial phase with gradient clipping"""    with tf.GradientTape() as tape:        real_pred = discriminator(real_sequences, training=True)        fake_pred = discriminator(fake_sequences, training=True)        real_labels = tf.ones_like(real_pred) * (1 - LABEL_SMOOTHING)        fake_labels = tf.ones_like(fake_pred) * LABEL_SMOOTHING        loss_real = bce_loss(real_labels, real_pred)        loss_fake = bce_loss(fake_labels, fake_pred)        loss = loss_real + loss_fake    gradients = tape.gradient(loss, discriminator.trainable_variables)        # Gradient clipping to prevent NaNs/explosions    gradients, grad_norm = tf.clip_by_global_norm(gradients, GRADIENT_CLIP)        # Check for NaNs and skip update if found    has_nan = tf.reduce_any([tf.reduce_any(tf.math.is_nan(g)) for g in gradients if g is not None])        if not has_nan:        optimizer_d.apply_gradients(zip(gradients, discriminator.trainable_variables))    # Keep accuracy computation on GPU (no .numpy() calls)    acc = (tf.reduce_mean(tf.cast(real_pred > 0.5, tf.float32)) +           tf.reduce_mean(tf.cast(fake_pred < 0.5, tf.float32))) / 2    return loss, loss_real, loss_fake, acc@tf.functiondef train_generator_adversarial(sequences, char_cond, scene_cond, temperature=1.0):    """Train generator with policy gradient and temperature annealing"""    with tf.GradientTape() as tape:        # Get logits with temperature scaling        logits = generator(sequences[:, :-1], training=True,                          char_condition=char_cond,                          scene_condition=scene_cond)                # Apply temperature to logits for exploration/exploitation        logits = logits / temperature        # Log probabilities        log_probs = tf.nn.log_softmax(logits, axis=-1)        # Get selected token log probs        batch_size = tf.shape(sequences)[0]        seq_len = tf.shape(sequences)[1] - 1        batch_idx = tf.tile(tf.expand_dims(tf.range(batch_size), 1), [1, seq_len])        time_idx = tf.tile(tf.expand_dims(tf.range(seq_len), 0), [batch_size, 1])        token_idx = sequences[:, 1:]        indices = tf.stack([batch_idx, time_idx, token_idx], axis=-1)        selected_log_probs = tf.gather_nd(log_probs, indices)        # Get rewards from discriminator        rewards = discriminator(sequences, training=False)        rewards = tf.squeeze(rewards, axis=-1)        # Advantages (simple baseline: mean reward)        baseline = tf.reduce_mean(rewards)        advantages = rewards - baseline        # Policy gradient loss        loss = -tf.reduce_mean(tf.reduce_sum(selected_log_probs, axis=1) * advantages)    gradients = tape.gradient(loss, generator.trainable_variables)        # Gradient clipping    gradients, grad_norm = tf.clip_by_global_norm(gradients, GRADIENT_CLIP)        # Check for NaNs    has_nan = tf.reduce_any([tf.reduce_any(tf.math.is_nan(g)) for g in gradients if g is not None])        if not has_nan:        optimizer_g.apply_gradients(zip(gradients, generator.trainable_variables))    return loss, tf.reduce_mean(rewards)print('Adversarial training step functions defined (OPTIMIZED)')print('✓ Gradient clipping enabled')print('✓ NaN checking enabled')print('✓ Temperature annealing support added')

In [ ]:
# Helper function to generate fake samples
def generate_fake_samples(batch_size, char_conditions, scene_conditions):
    """Generate fake samples from generator"""
    # Start with random token or padding
    start_tokens = tf.ones((batch_size, 1), dtype=tf.int32)

    # Generate sequences
    generated = generator.generate_sequence(
        start_tokens,
        max_length=SEQ_LENGTH,
        temperature=1.0,
        char_condition=char_conditions,
        scene_condition=scene_conditions,
        method='sampling'
    )

    return generated

# Create tf.data datasets for efficient batching
train_dataset = tf.data.Dataset.from_tensor_slices((
    X_train, char_train, scene_train
)).shuffle(10000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((
    X_val, char_val, scene_val
)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print(f'Training dataset: {len(list(train_dataset))} batches')
print(f'Validation dataset: {len(list(val_dataset))} batches')
print('\nReady to start training!')

In [ ]:
# Phase 1: Pre-train Generator (OPTIMIZED)print('='*80)print('PHASE 1: GENERATOR PRE-TRAINING (OPTIMIZED)')print('='*80)import timefor epoch in range(PRETRAIN_EPOCHS_G):    print(f'\nEpoch {epoch+1}/{PRETRAIN_EPOCHS_G}')    epoch_losses_tensor = []    epoch_start_time = time.time()    for batch_idx, (batch_seqs, batch_char, batch_scene) in enumerate(train_dataset):        loss = pretrain_generator_step(batch_seqs, batch_char, batch_scene)        epoch_losses_tensor.append(loss)  # Keep as tensor        if (batch_idx + 1) % 20 == 0:            print(f'  Batch {batch_idx+1}: Loss={loss.numpy():.4f}')        # Convert to numpy ONCE per epoch    epoch_losses = [l.numpy() for l in epoch_losses_tensor]    avg_loss = np.mean(epoch_losses)    epoch_time = time.time() - epoch_start_time        print(f'  Average Loss: {avg_loss:.4f} | Time: {epoch_time:.2f}s')print('Generator pre-training complete!')

In [ ]:
# Phase 2: Pre-train Discriminator (OPTIMIZED)print('='*80)print('PHASE 2: DISCRIMINATOR PRE-TRAINING (OPTIMIZED)')print('='*80)import timefor epoch in range(PRETRAIN_EPOCHS_D):    print(f'\nEpoch {epoch+1}/{PRETRAIN_EPOCHS_D}')    epoch_losses_tensor = []    epoch_accs_tensor = []    epoch_start_time = time.time()    for batch_idx, (batch_seqs, batch_char, batch_scene) in enumerate(train_dataset):        # Generate fake samples        fake_seqs = generate_fake_samples(            tf.shape(batch_seqs)[0], batch_char, batch_scene        )        # Train discriminator        loss, acc = pretrain_discriminator_step(batch_seqs, fake_seqs)                # Accumulate as tensors        epoch_losses_tensor.append(loss)        epoch_accs_tensor.append(acc)        if (batch_idx + 1) % 20 == 0:            print(f'  Batch {batch_idx+1}: Loss={loss.numpy():.4f}, Acc={acc.numpy():.4f}')        # Convert to numpy ONCE per epoch    epoch_losses = [l.numpy() for l in epoch_losses_tensor]    epoch_accs = [a.numpy() for a in epoch_accs_tensor]    avg_loss = np.mean(epoch_losses)    avg_acc = np.mean(epoch_accs)    epoch_time = time.time() - epoch_start_time        print(f'  Average Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f} | Time: {epoch_time:.2f}s')print('Discriminator pre-training complete!')

In [ ]:
# Phase 3: Adversarial Training (OPTIMIZED)print('='*80)print('PHASE 3: ADVERSARIAL TRAINING (OPTIMIZED)')print('='*80)print('🚀 Performance optimizations enabled:')print('  ✓ No .numpy() calls in inner loop')print('  ✓ GPU tensor accumulation')print('  ✓ Temperature annealing')print('  ✓ Gradient clipping with NaN protection')print('  ✓ Batch timing for speed comparison')print('='*80)import time# Initialize temperaturecurrent_temp = TEMP_START# Track batch times for performance measurementbatch_times = []for epoch in range(ADVERSARIAL_EPOCHS):    print(f'\nEpoch {epoch+1}/{ADVERSARIAL_EPOCHS} | Temperature: {current_temp:.3f}')        # Use TensorFlow tensors for accumulation (stay on GPU)    g_losses_tensor = []    d_losses_tensor = []    d_accs_tensor = []    g_rewards_tensor = []        epoch_start_time = time.time()    batch_count = 0    for batch_idx, (batch_seqs, batch_char, batch_scene) in enumerate(train_dataset):        batch_start_time = time.time()                # Train Discriminator (with optional interval training)        if batch_idx % D_TRAIN_INTERVAL == 0:            for _ in range(D_STEPS):                fake_seqs = generate_fake_samples(                    tf.shape(batch_seqs)[0], batch_char, batch_scene                )                d_loss, d_loss_real, d_loss_fake, d_acc = train_discriminator_adversarial(                    batch_seqs, fake_seqs                )                # Accumulate as tensors (NO .numpy() calls in loop!)        d_losses_tensor.append(d_loss)        d_accs_tensor.append(d_acc)                # Train Generator with temperature annealing        for _ in range(G_STEPS):            g_loss, g_reward = train_generator_adversarial(                batch_seqs, batch_char, batch_scene, temperature=current_temp            )                # Accumulate as tensors        g_losses_tensor.append(g_loss)        g_rewards_tensor.append(g_reward)                batch_time = time.time() - batch_start_time        batch_times.append(batch_time)        batch_count += 1                # Print progress every 10 batches (convert to numpy ONLY for display)        if (batch_idx + 1) % 10 == 0:            # Single conversion per metric for display            g_loss_val = g_loss.numpy()            d_loss_val = d_loss.numpy()            d_acc_val = d_acc.numpy()            g_reward_val = g_reward.numpy()            avg_batch_time = sum(batch_times[-10:]) / min(10, len(batch_times))                        print(f'  Batch {batch_idx+1}: '                  f'G_Loss={g_loss_val:.4f}, '                  f'D_Loss={d_loss_val:.4f}, '                  f'D_Acc={d_acc_val:.4f}, '                  f'Reward={g_reward_val:.4f} '                  f'| ⚡{avg_batch_time*1000:.1f}ms/batch')        # Convert accumulated tensors to numpy ONCE per epoch    g_losses = [loss.numpy() for loss in g_losses_tensor]    d_losses = [loss.numpy() for loss in d_losses_tensor]    d_accs = [acc.numpy() for acc in d_accs_tensor]    g_rewards = [reward.numpy() for reward in g_rewards_tensor]        # Epoch summary    avg_g_loss = np.mean(g_losses)    avg_d_loss = np.mean(d_losses)    avg_d_acc = np.mean(d_accs)    avg_reward = np.mean(g_rewards)        epoch_time = time.time() - epoch_start_time    avg_batch_time = epoch_time / batch_count if batch_count > 0 else 0    history['g_loss'].append(avg_g_loss)    history['d_loss'].append(avg_d_loss)    history['d_acc'].append(avg_d_acc)    history['g_reward'].append(avg_reward)    history['epoch'].append(epoch + 1)        # Anneal temperature    current_temp = max(TEMP_END, current_temp * TEMP_DECAY)    print(f'\n  📊 EPOCH SUMMARY:')    print(f'    Generator Loss: {avg_g_loss:.4f}')    print(f'    Discriminator Loss: {avg_d_loss:.4f}')    print(f'    Discriminator Accuracy: {avg_d_acc:.4f} {"🎯" if avg_d_acc > 0.7 else ""}')    print(f'    Average Reward: {avg_reward:.4f}')    print(f'    Epoch Time: {epoch_time:.2f}s | Avg Batch: {avg_batch_time*1000:.1f}ms')    # Generate sample after each epoch    if (epoch + 1) % 5 == 0:        print(f'\n  🎭 Sample Generation (Temperature={current_temp:.2f}):')        test_start = tf.ones((1, 1), dtype=tf.int32)        test_char = tf.constant([0])  # First character        test_scene = tf.constant([0])  # First scene        sample = generator.generate_sequence(            test_start, max_length=30, temperature=current_temp,            char_condition=test_char, scene_condition=test_scene        )        tokens = sample[0].numpy()        decoded = ' '.join([tokenizer.index_word.get(int(t), '<UNK>')                           for t in tokens if t > 0])        print(f'    "{decoded}"')print('\n' + '='*80)print('✅ ADVERSARIAL TRAINING COMPLETE!')print('='*80)print(f'Performance Stats:')print(f'  Total batches: {len(batch_times)}')print(f'  Avg batch time: {np.mean(batch_times)*1000:.1f}ms')print(f'  Min batch time: {np.min(batch_times)*1000:.1f}ms')print(f'  Max batch time: {np.max(batch_times)*1000:.1f}ms')print(f'  Final temperature: {current_temp:.3f}')print(f'  Final D accuracy: {history["d_acc"][-1]:.4f}')print('='*80)

## Section 5.0: Script Autocomplete Tool

Now we'll create an interactive dialogue autocomplete system.

### Features:

**1. autocomplete_dialogue() Function**:
- Takes seed text, character, scene type, temperature
- Generates multiple completions
- Returns ranked results with quality scores

**Parameters**:
- `seed_text`: Starting dialogue (e.g., "I will destroy all")
- `character`: Character name (Eren, Mikasa, Armin, etc.)
- `scene_type`: Scene context (Battle, Training, Strategy, etc.)
- `num_completions`: Number of different completions to generate
- `max_length`: Maximum tokens in completion
- `temperature`: Sampling randomness (0.5-2.0)

**2. Interactive Widget Demo**:
- Text input for seed dialogue
- Dropdown for character selection
- Dropdown for scene selection
- Slider for temperature control
- Button to generate completions
- Display with quality scores from discriminator

**3. Quality Scoring**:
- Use discriminator to rate each completion
- Higher score = more realistic
- Sort completions by quality
- Show top results

### Use Cases:

- **Script writing assistance**: Complete dialogue lines
- **Creative exploration**: Generate variations
- **Character voice**: Ensure consistent character speech
- **Scene coherence**: Match dialogue to scene type

In [ ]:
def autocomplete_dialogue(seed_text, character=None, scene_type=None,
                          num_completions=5, max_length=50, temperature=0.8):
    """
    Generate dialogue completions with quality scores

    Args:
        seed_text: Starting text to complete
        character: Character name
        scene_type: Scene type
        num_completions: Number of completions to generate
        max_length: Maximum length of completion
        temperature: Sampling temperature

    Returns:
        List of (completion_text, quality_score) tuples
    """
    # Encode seed text
    seed_tokens = tokenizer.texts_to_sequences([seed_text.lower()])[0]
    if not seed_tokens:
        seed_tokens = [1]  # Start token

    # Pad to at least length 1
    seed_tokens = seed_tokens[:max_length]
    seed_tensor = tf.constant([seed_tokens])

    # Get character and scene indices
    char_idx = char_to_idx.get(character, 0)
    scene_idx = scene_to_idx.get(scene_type, 0)

    char_tensor = tf.constant([char_idx])
    scene_tensor = tf.constant([scene_idx])

    # Generate multiple completions
    completions = []

    for i in range(num_completions):
        # Generate sequence
        generated = generator.generate_sequence(
            seed_tensor,
            max_length=max_length,
            temperature=temperature,
            char_condition=char_tensor,
            scene_condition=scene_tensor,
            method='sampling'
        )

        # Get quality score from discriminator
        # Pad/truncate to SEQ_LENGTH
        gen_padded = pad_sequences(
            generated.numpy(),
            maxlen=SEQ_LENGTH,
            padding='pre'
        )
        quality_score = discriminator(gen_padded, training=False).numpy()[0][0]

        # Decode tokens
        tokens = generated[0].numpy()
        decoded = ' '.join([tokenizer.index_word.get(int(t), '')
                           for t in tokens if t > 0])
        decoded = decoded.strip()

        completions.append((decoded, float(quality_score)))

    # Sort by quality score (descending)
    completions.sort(key=lambda x: x[1], reverse=True)

    return completions

print('autocomplete_dialogue() function defined')

In [ ]:
# Test the autocomplete function
print('='*80)
print('TESTING AUTOCOMPLETE FUNCTION')
print('='*80)

test_cases = [
    ('I will destroy', 'Eren', 'Battle', 0.8),
    ('We need to', 'Armin', 'Strategy Meeting', 0.7),
    ('I will protect', 'Mikasa', 'Battle', 0.8),
]

for seed, char, scene, temp in test_cases:
    print(f'\nSeed: "{seed}" | Character: {char} | Scene: {scene} | Temp: {temp}')
    print('-' * 80)

    completions = autocomplete_dialogue(
        seed, char, scene,
        num_completions=3,
        max_length=30,
        temperature=temp
    )

    for idx, (text, score) in enumerate(completions, 1):
        print(f'{idx}. [{score:.3f}] {text}')

print('\n' + '='*80)

In [ ]:
# Interactive widget demo
try:
    from ipywidgets import interact, widgets
    from IPython.display import display, HTML

    # Widget components
    seed_input = widgets.Text(
        value='I will',
        placeholder='Enter seed text...',
        description='Seed Text:',
        style={'description_width': 'initial'}
    )

    char_dropdown = widgets.Dropdown(
        options=list(char_to_idx.keys()),
        value=list(char_to_idx.keys())[0],
        description='Character:',
        style={'description_width': 'initial'}
    )

    scene_dropdown = widgets.Dropdown(
        options=list(scene_to_idx.keys()),
        value=list(scene_to_idx.keys())[0],
        description='Scene:',
        style={'description_width': 'initial'}
    )

    temp_slider = widgets.FloatSlider(
        value=0.8,
        min=0.3,
        max=1.5,
        step=0.1,
        description='Temperature:',
        style={'description_width': 'initial'}
    )

    num_slider = widgets.IntSlider(
        value=5,
        min=1,
        max=10,
        description='# Completions:',
        style={'description_width': 'initial'}
    )

    output = widgets.Output()

    def generate_button_clicked(b):
        with output:
            output.clear_output()
            print('Generating completions...')

            completions = autocomplete_dialogue(
                seed_input.value,
                char_dropdown.value,
                scene_dropdown.value,
                num_completions=num_slider.value,
                max_length=40,
                temperature=temp_slider.value
            )

            output.clear_output()
            print(f'Generated {len(completions)} completions:\n')
            print('='*80)

            for idx, (text, score) in enumerate(completions, 1):
                quality = '★' * int(score * 5)
                print(f'\n{idx}. Quality: {quality} ({score:.3f})')
                print(f'   {text}')

            print('\n' + '='*80)

    generate_btn = widgets.Button(
        description='Generate Dialogue',
        button_style='success',
        icon='play'
    )
    generate_btn.on_click(generate_button_clicked)

    # Display widgets
    print('='*80)
    print('INTERACTIVE DIALOGUE AUTOCOMPLETE')
    print('='*80)

    display(HTML('<h3>Custom Dialogue Dialogue Generator</h3>'))
    display(seed_input)
    display(char_dropdown)
    display(scene_dropdown)
    display(temp_slider)
    display(num_slider)
    display(generate_btn)
    display(output)

except ImportError:
    print('ipywidgets not available. Use autocomplete_dialogue() function directly.')


## Section 6.0: Evaluation Metrics

We evaluate our GAN using multiple metrics:

### 1. Perplexity
Measures how well the model predicts the test set:
$$PPL = \exp\left(-\frac{1}{N}\sum_{i=1}^{N} \log P(w_i|w_{<i})\right)$$

Lower is better. Typical ranges:
- Excellent: < 20
- Good: 20-50
- Fair: 50-100
- Poor: > 100

### 2. BLEU Score
Measures n-gram overlap with reference text:
$$BLEU = BP \cdot \exp\left(\sum_{n=1}^{N} w_n \log p_n\right)$$

Where:
- $p_n$: n-gram precision
- $BP$: Brevity penalty
- Range: [0, 1], higher is better

### 3. Self-BLEU (Diversity)
Measures diversity by computing BLEU between generated samples:
- Lower Self-BLEU = Higher diversity
- Detects mode collapse
- Ensures variety in generation

### 4. Discriminator Accuracy
How well D distinguishes real vs fake:
- Target: ~50-70% (balanced)
- Too high (>90%): Generator failing
- Too low (<40%): Discriminator failing

### 5. Generator Loss Curves
Monitor training stability:
- Should gradually decrease
- Oscillations are normal
- Divergence indicates instability

### 6. Sample Quality (Human Evaluation)
Ultimate test:
- Fluency: Grammatically correct?
- Coherence: Makes sense in context?
- Character voice: Sounds like the character?
- Diversity: Varied or repetitive?

In [ ]:
def compute_perplexity(model, dataset, tokenizer_obj):
    """Compute perplexity on dataset"""
    total_loss = 0
    total_tokens = 0

    for batch_seqs, batch_char, batch_scene in dataset:
        logits = model(batch_seqs[:, :-1], training=False,
                      char_condition=batch_char,
                      scene_condition=batch_scene)

        loss = sparse_ce_loss(batch_seqs[:, 1:], logits)

        # Count non-padding tokens
        mask = tf.cast(batch_seqs[:, 1:] != 0, tf.float32)
        num_tokens = tf.reduce_sum(mask)

        total_loss += loss.numpy() * num_tokens.numpy()
        total_tokens += num_tokens.numpy()

    avg_loss = total_loss / total_tokens
    perplexity = np.exp(avg_loss)

    return perplexity

def compute_bleu(reference, hypothesis):
    """Compute BLEU score"""
    smoothing = SmoothingFunction()
    reference_tokens = reference.lower().split()
    hypothesis_tokens = hypothesis.lower().split()

    score = sentence_bleu(
        [reference_tokens],
        hypothesis_tokens,
        smoothing_function=smoothing.method1
    )
    return score

def compute_self_bleu(generated_texts, sample_size=100):
    """Compute Self-BLEU for diversity measurement"""
    if len(generated_texts) < 2:
        return 0.0

    # Sample if too many
    if len(generated_texts) > sample_size:
        generated_texts = np.random.choice(
            generated_texts, sample_size, replace=False
        ).tolist()

    smoothing = SmoothingFunction()
    scores = []

    for i, hyp in enumerate(generated_texts):
        refs = [text for j, text in enumerate(generated_texts) if j != i]
        hyp_tokens = hyp.lower().split()
        ref_tokens = [ref.lower().split() for ref in refs]

        if hyp_tokens:
            score = sentence_bleu(
                ref_tokens,
                hyp_tokens,
                smoothing_function=smoothing.method1
            )
            scores.append(score)

    return np.mean(scores) if scores else 0.0

print('Evaluation metric functions defined')

In [ ]:
# Evaluate models
print('='*80)
print('EVALUATION METRICS')
print('='*80)

# 1. Perplexity
print('\n1. Computing Perplexity...')
val_perplexity = compute_perplexity(generator, val_dataset, tokenizer)
print(f'   Validation Perplexity: {val_perplexity:.2f}')

# 2. Generate samples for BLEU and Self-BLEU
print('\n2. Generating samples for BLEU evaluation...')
generated_samples = []
reference_samples = []

for i in range(50):
    # Random character and scene
    char_idx = np.random.randint(0, num_characters)
    scene_idx = np.random.randint(0, num_scenes)

    # Generate
    start = tf.ones((1, 1), dtype=tf.int32)
    generated = generator.generate_sequence(
        start, max_length=30, temperature=0.8,
        char_condition=tf.constant([char_idx]),
        scene_condition=tf.constant([scene_idx])
    )

    tokens = generated[0].numpy()
    text = ' '.join([tokenizer.index_word.get(int(t), '')
                    for t in tokens if t > 0])
    generated_samples.append(text.strip())

    # Get reference from validation set
    if i < len(X_val):
        ref_tokens = X_val[i]
        ref_text = ' '.join([tokenizer.index_word.get(int(t), '')
                            for t in ref_tokens if t > 0])
        reference_samples.append(ref_text.strip())

# 3. BLEU Score
print('\n3. Computing BLEU scores...')
bleu_scores = []
for gen, ref in zip(generated_samples[:len(reference_samples)], reference_samples):
    if gen and ref:
        score = compute_bleu(ref, gen)
        bleu_scores.append(score)

avg_bleu = np.mean(bleu_scores) if bleu_scores else 0.0
print(f'   Average BLEU Score: {avg_bleu:.4f}')

# 4. Self-BLEU (Diversity)
print('\n4. Computing Self-BLEU (diversity metric)...')
self_bleu = compute_self_bleu(generated_samples)
print(f'   Self-BLEU Score: {self_bleu:.4f}')
print(f'   (Lower is better - indicates higher diversity)')

# 5. Discriminator Accuracy
print('\n5. Discriminator Accuracy...')
if history['d_acc']:
    final_d_acc = history['d_acc'][-1]
    print(f'   Final Discriminator Accuracy: {final_d_acc:.4f}')
    print(f'   (Target range: 0.50-0.70 for balanced training)')

# 6. Sample Generation at Different Temperatures
print('\n6. Sample Generations at Different Temperatures:')
print('-' * 80)

for temp in [0.5, 0.8, 1.0, 1.2]:
    start = tf.ones((1, 1), dtype=tf.int32)
    generated = generator.generate_sequence(
        start, max_length=25, temperature=temp,
        char_condition=tf.constant([0]),
        scene_condition=tf.constant([0])
    )

    tokens = generated[0].numpy()
    text = ' '.join([tokenizer.index_word.get(int(t), '')
                    for t in tokens if t > 0])

    print(f'\nTemperature {temp}:')
    print(f'  "{text}"')

print('\n' + '='*80)
print('EVALUATION SUMMARY')
print('='*80)
print(f'Perplexity: {val_perplexity:.2f}')
print(f'BLEU Score: {avg_bleu:.4f}')
print(f'Self-BLEU (Diversity): {self_bleu:.4f}')
if history['d_acc']:
    print(f'Discriminator Accuracy: {final_d_acc:.4f}')
print('='*80)

## Section 7.0: Advanced Features

### 1. Character-Conditioned Generation

Generate dialogue specific to characters:
- **Eren**: Passionate, determined, emotional
- **Mikasa**: Protective, calm, focused
- **Armin**: Strategic, thoughtful, analytical
- **Levi**: Stern, commanding, pragmatic

### 2. Scene-Type Conditioning

Adapt dialogue to scene context:
- **Scene Type 1**: Define based on your dataset
- **Scene Type 2**: Define based on your dataset
- **Scene Type 3**: Define based on your dataset
- Additional scene types as needed

### 3. Length Control

Generate sequences of specific lengths:
- Short responses (5-10 tokens)
- Medium dialogue (10-30 tokens)
- Long passages (30+ tokens)

### 4. Beam Search Decoding

Alternative to sampling for more consistent output:
- Keep top-k candidates at each step
- Select best overall sequence
- More deterministic than sampling

### 5. Top-K and Top-P (Nucleus) Sampling

**Top-K Sampling**:
- Sample from top k most likely tokens
- k=50 is common
- Filters out unlikely tokens

**Top-P (Nucleus) Sampling**:
- Sample from smallest set with cumulative probability > p
- p=0.9 is common
- Adaptive vocabulary size

In [ ]:
def generate_character_dialogue(character, scene, seed='', temperature=0.8,
                                max_length=30, method='sampling'):
    """
    Generate dialogue for specific character in specific scene

    Args:
        character: Character name
        scene: Scene type
        seed: Optional seed text
        temperature: Sampling temperature
        max_length: Maximum tokens
        method: 'sampling' or 'greedy'

    Returns:
        Generated dialogue text
    """
    # Encode seed if provided
    if seed:
        seed_tokens = tokenizer.texts_to_sequences([seed.lower()])[0]
        start_tensor = tf.constant([seed_tokens])
    else:
        start_tensor = tf.ones((1, 1), dtype=tf.int32)

    # Get conditions
    char_idx = char_to_idx.get(character, 0)
    scene_idx = scene_to_idx.get(scene, 0)

    # Generate
    generated = generator.generate_sequence(
        start_tensor,
        max_length=max_length,
        temperature=temperature,
        char_condition=tf.constant([char_idx]),
        scene_condition=tf.constant([scene_idx]),
        method=method
    )

    # Decode
    tokens = generated[0].numpy()
    text = ' '.join([tokenizer.index_word.get(int(t), '')
                    for t in tokens if t > 0])

    return text.strip()

# Demonstrate character-specific generation
print('='*80)
print('CHARACTER-CONDITIONED GENERATION')
print('='*80)

characters_to_test = list(char_to_idx.keys())[:4]  # Test first 4 characters
scene = list(scene_to_idx.keys())[0]  # Use first scene

for character in characters_to_test:
    print(f'\n{character} (Scene: {scene}):')
    for i in range(3):
        dialogue = generate_character_dialogue(
            character, scene, temperature=0.8
        )
        print(f'  {i+1}. "{dialogue}"')

print('\n' + '='*80)

In [ ]:
# Demonstrate scene-conditioned generation
print('='*80)
print('SCENE-CONDITIONED GENERATION')
print('='*80)

character = list(char_to_idx.keys())[0]  # Use first character
scenes_to_test = list(scene_to_idx.keys())[:4]  # Test first 4 scenes

for scene in scenes_to_test:
    print(f'\nScene: {scene} (Character: {character}):')
    for i in range(2):
        dialogue = generate_character_dialogue(
            character, scene, temperature=0.8
        )
        print(f'  {i+1}. "{dialogue}"')

print('\n' + '='*80)

In [ ]:
# Length-controlled generation
print('='*80)
print('LENGTH-CONTROLLED GENERATION')
print('='*80)

character = list(char_to_idx.keys())[0]
scene = list(scene_to_idx.keys())[0]

for length in [10, 20, 30, 40]:
    dialogue = generate_character_dialogue(
        character, scene, temperature=0.8, max_length=length
    )
    print(f'\nMax Length {length} tokens:')
    print(f'  "{dialogue}"')
    print(f'  (Actual length: {len(dialogue.split())} words)')

print('\n' + '='*80)

## Section 8.0: Visualizations

Visualize training dynamics and model behavior:

### 1. Training Loss Curves
- Generator loss over epochs
- Discriminator loss over epochs
- Shows training stability and convergence

### 2. Discriminator Accuracy
- Track D's ability to distinguish real vs fake
- Target: 50-70% (balanced)
- Too high/low indicates training issues

### 3. Generator Reward
- Average discriminator score for generated samples
- Should increase over training
- Indicates improving generation quality

### 4. t-SNE Embedding Visualization
- Project real and generated sequences to 2D
- Visualize distribution overlap
- Good GAN: distributions overlap

### 5. Word Clouds
- Real data word frequency
- Generated data word frequency
- Compare vocabulary usage

In [ ]:
# 1. Training Loss Curves
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Generator Loss
if history['g_loss']:
    axes[0, 0].plot(history['epoch'], history['g_loss'], 'b-', linewidth=2)
    axes[0, 0].set_title('Generator Loss', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].grid(True, alpha=0.3)

# Discriminator Loss
if history['d_loss']:
    axes[0, 1].plot(history['epoch'], history['d_loss'], 'r-', linewidth=2)
    axes[0, 1].set_title('Discriminator Loss', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].grid(True, alpha=0.3)

# Discriminator Accuracy
if history['d_acc']:
    axes[1, 0].plot(history['epoch'], history['d_acc'], 'g-', linewidth=2)
    axes[1, 0].axhline(y=0.5, color='k', linestyle='--', alpha=0.5, label='Random (0.5)')
    axes[1, 0].axhline(y=0.7, color='orange', linestyle='--', alpha=0.5, label='Target Max (0.7)')
    axes[1, 0].set_title('Discriminator Accuracy', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Accuracy')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].set_ylim([0, 1])

# Generator Reward (Discriminator score)
if history['g_reward']:
    axes[1, 1].plot(history['epoch'], history['g_reward'], 'm-', linewidth=2)
    axes[1, 1].set_title('Generator Reward (Avg. D Score)', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Reward')
    axes[1, 1].grid(True, alpha=0.3)
    axes[1, 1].set_ylim([0, 1])

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print('Training curves plotted and saved to training_curves.png')

In [ ]:
# 2. t-SNE Visualization of Real vs Generated Embeddings
print('Generating t-SNE visualization...')

# Get real samples
num_samples = min(200, len(X_val))
real_samples = X_val[:num_samples]

# Generate fake samples
fake_samples = []
for i in range(num_samples):
    char_idx = np.random.randint(0, num_characters)
    scene_idx = np.random.randint(0, num_scenes)

    start = tf.ones((1, 1), dtype=tf.int32)
    generated = generator.generate_sequence(
        start, max_length=SEQ_LENGTH, temperature=0.8,
        char_condition=tf.constant([char_idx]),
        scene_condition=tf.constant([scene_idx])
    )

    # Pad to SEQ_LENGTH
    gen_padded = pad_sequences(
        generated.numpy(),
        maxlen=SEQ_LENGTH,
        padding='pre'
    )
    fake_samples.append(gen_padded[0])

fake_samples = np.array(fake_samples)

# Extract features using discriminator
real_features = discriminator.get_features(real_samples).numpy()
fake_features = discriminator.get_features(fake_samples).numpy()

# Combine for t-SNE
all_features = np.vstack([real_features, fake_features])
labels = ['Real'] * len(real_features) + ['Generated'] * len(fake_features)

# Apply t-SNE
print('Applying t-SNE (this may take a moment)...')
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
embedded = tsne.fit_transform(all_features)

# Plot
plt.figure(figsize=(12, 8))
real_embedded = embedded[:len(real_features)]
fake_embedded = embedded[len(real_features):]

plt.scatter(real_embedded[:, 0], real_embedded[:, 1],
           c='blue', alpha=0.6, s=50, label='Real Dialogue', edgecolors='k', linewidth=0.5)
plt.scatter(fake_embedded[:, 0], fake_embedded[:, 1],
           c='red', alpha=0.6, s=50, label='Generated Dialogue', edgecolors='k', linewidth=0.5)

plt.title('t-SNE: Real vs Generated Dialogue Embeddings', fontsize=16, fontweight='bold')
plt.xlabel('t-SNE Dimension 1', fontsize=12)
plt.ylabel('t-SNE Dimension 2', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('tsne_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print('t-SNE visualization saved to tsne_visualization.png')

In [ ]:
# 3. Word Clouds for Real vs Generated Text
print('Creating word clouds...')

# Real text
real_text = ' '.join([
    ' '.join([tokenizer.index_word.get(int(t), '') for t in seq if t > 0])
    for seq in X_val[:500]
])

# Generated text
generated_texts = []
for i in range(100):
    char_idx = np.random.randint(0, num_characters)
    scene_idx = np.random.randint(0, num_scenes)
    text = generate_character_dialogue(
        list(char_to_idx.keys())[char_idx],
        list(scene_to_idx.keys())[scene_idx],
        temperature=0.8
    )
    generated_texts.append(text)

generated_text = ' '.join(generated_texts)

# Create word clouds
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Real data word cloud
wordcloud_real = WordCloud(
    width=800, height=400,
    background_color='white',
    colormap='Blues',
    max_words=100
).generate(real_text)

axes[0].imshow(wordcloud_real, interpolation='bilinear')
axes[0].set_title('Real Dialogue Word Cloud', fontsize=14, fontweight='bold')
axes[0].axis('off')

# Generated data word cloud
wordcloud_fake = WordCloud(
    width=800, height=400,
    background_color='white',
    colormap='Reds',
    max_words=100
).generate(generated_text)

axes[1].imshow(wordcloud_fake, interpolation='bilinear')
axes[1].set_title('Generated Dialogue Word Cloud', fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.savefig('wordclouds.png', dpi=150, bbox_inches='tight')
plt.show()

print('Word clouds saved to wordclouds.png')

In [ ]:
# 4. Combined Metrics Summary Plot
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# Generator and Discriminator Losses
ax1 = fig.add_subplot(gs[0, :])
if history['g_loss'] and history['d_loss']:
    ax1.plot(history['epoch'], history['g_loss'], 'b-', linewidth=2, label='Generator Loss', marker='o')
    ax1_twin = ax1.twinx()
    ax1_twin.plot(history['epoch'], history['d_loss'], 'r-', linewidth=2, label='Discriminator Loss', marker='s')
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Generator Loss', fontsize=12, color='b')
    ax1_twin.set_ylabel('Discriminator Loss', fontsize=12, color='r')
    ax1.tick_params(axis='y', labelcolor='b')
    ax1_twin.tick_params(axis='y', labelcolor='r')
    ax1.set_title('Training Losses Over Time', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend(loc='upper left')
    ax1_twin.legend(loc='upper right')

# Discriminator Accuracy with zones
ax2 = fig.add_subplot(gs[1, 0])
if history['d_acc']:
    ax2.plot(history['epoch'], history['d_acc'], 'g-', linewidth=2, marker='o')
    ax2.axhspan(0.5, 0.7, alpha=0.2, color='green', label='Target Zone')
    ax2.axhline(y=0.5, color='k', linestyle='--', alpha=0.5)
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Accuracy', fontsize=12)
    ax2.set_title('Discriminator Accuracy', fontsize=14, fontweight='bold')
    ax2.set_ylim([0, 1])
    ax2.grid(True, alpha=0.3)
    ax2.legend()

# Generator Reward
ax3 = fig.add_subplot(gs[1, 1])
if history['g_reward']:
    ax3.plot(history['epoch'], history['g_reward'], 'm-', linewidth=2, marker='o')
    ax3.set_xlabel('Epoch', fontsize=12)
    ax3.set_ylabel('Average Reward', fontsize=12)
    ax3.set_title('Generator Quality (Discriminator Score)', fontsize=14, fontweight='bold')
    ax3.set_ylim([0, 1])
    ax3.grid(True, alpha=0.3)

# Metrics Summary Table
ax4 = fig.add_subplot(gs[2, :])
ax4.axis('off')

metrics_data = [
    ['Metric', 'Value', 'Status'],
    ['Validation Perplexity', f'{val_perplexity:.2f}', '✓' if val_perplexity < 100 else '⚠'],
    ['BLEU Score', f'{avg_bleu:.4f}', '✓' if avg_bleu > 0.1 else '⚠'],
    ['Self-BLEU (Diversity)', f'{self_bleu:.4f}', '✓' if self_bleu < 0.8 else '⚠'],
]

if history['d_acc']:
    metrics_data.append(['Final D Accuracy', f'{final_d_acc:.4f}',
                        '✓' if 0.5 <= final_d_acc <= 0.7 else '⚠'])

table = ax4.table(cellText=metrics_data, cellLoc='center', loc='center',
                 colWidths=[0.3, 0.2, 0.1])
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1, 2)

# Style header row
for i in range(3):
    table[(0, i)].set_facecolor('#4CAF50')
    table[(0, i)].set_text_props(weight='bold', color='white')

ax4.set_title('Evaluation Metrics Summary', fontsize=14, fontweight='bold', pad=20)

plt.savefig('comprehensive_metrics.png', dpi=150, bbox_inches='tight')
plt.show()

print('Comprehensive metrics visualization saved to comprehensive_metrics.png')

## Section 9.0: Model Persistence

Save and load trained models for future use.

### What to Save:

1. **Generator Model**: Architecture + weights
2. **Discriminator Model**: Architecture + weights
3. **Tokenizer**: Vocabulary and token mappings
4. **Metadata**: Character/scene mappings, hyperparameters
5. **Training History**: Losses, accuracies, metrics

### Storage Location:

Models saved to: `/content/drive/MyDrive/DLA_Notebooks_Data_PGPM/`

### File Structure:
```
DLA_Notebooks_Data_PGPM/
├── generator/
│   ├── saved_model.pb
│   └── variables/
├── discriminator/
│   ├── saved_model.pb
│   └── variables/
├── tokenizer.pickle
├── metadata.json
└── training_history.json
```

### Loading Models:

```python
# Load generator
generator = keras.models.load_model('path/to/generator')

# Load tokenizer
with open('path/to/tokenizer.pickle', 'rb') as f:
    tokenizer = pickle.load(f)
```

In [ ]:
# Create save directory
SAVE_DIR = '/content/drive/MyDrive/DLA_Notebooks_Data_PGPM/'

# Try to create directory (will fail if not in Colab with Drive mounted)
try:
    import os
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'Save directory: {SAVE_DIR}')
except:
    # Fallback to local directory
    SAVE_DIR = './models/'
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f'Using local save directory: {SAVE_DIR}')
    print('(Mount Google Drive in Colab to use persistent storage)')


In [ ]:
def save_models(save_dir=SAVE_DIR):
    """Save all models and associated data"""
    print('='*80)
    print('SAVING MODELS AND DATA')
    print('='*80)

    # Save generator
    gen_path = os.path.join(save_dir, 'generator')
    generator.save(gen_path)
    print(f'✓ Generator saved to: {gen_path}')

    # Save discriminator
    disc_path = os.path.join(save_dir, 'discriminator')
    discriminator.save(disc_path)
    print(f'✓ Discriminator saved to: {disc_path}')

    # Save tokenizer
    tokenizer_path = os.path.join(save_dir, 'tokenizer.pickle')
    with open(tokenizer_path, 'wb') as f:
        pickle.dump(tokenizer, f)
    print(f'✓ Tokenizer saved to: {tokenizer_path}')

    # Save metadata
    metadata = {
        'vocab_size': data_processor.vocab_size,
        'seq_length': SEQ_LENGTH,
        'char_to_idx': char_to_idx,
        'idx_to_char': idx_to_char,
        'scene_to_idx': scene_to_idx,
        'idx_to_scene': idx_to_scene,
        'num_characters': num_characters,
        'num_scenes': num_scenes,
        'hyperparameters': {
            'embedding_dim': 256,
            'lstm_units': 512,
            'num_lstm_layers': 2,
            'batch_size': BATCH_SIZE,
            'learning_rate_g': LEARNING_RATE_G,
            'learning_rate_d': LEARNING_RATE_D,
        }
    }

    metadata_path = os.path.join(save_dir, 'metadata.json')
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f'✓ Metadata saved to: {metadata_path}')

    # Save training history
    history_path = os.path.join(save_dir, 'training_history.json')
    with open(history_path, 'w') as f:
        # Convert numpy types to Python types
        history_serializable = {
            k: [float(v) for v in vals] if isinstance(vals, list) else vals
            for k, vals in history.items()
        }
        json.dump(history_serializable, f, indent=2)
    print(f'✓ Training history saved to: {history_path}')

    print('='*80)
    print('ALL MODELS AND DATA SAVED SUCCESSFULLY!')
    print('='*80)

def load_models(save_dir=SAVE_DIR):
    """Load all models and associated data"""
    print('='*80)
    print('LOADING MODELS AND DATA')
    print('='*80)

    # Load generator
    gen_path = os.path.join(save_dir, 'generator')
    loaded_generator = keras.models.load_model(gen_path)
    print(f'✓ Generator loaded from: {gen_path}')

    # Load discriminator
    disc_path = os.path.join(save_dir, 'discriminator')
    loaded_discriminator = keras.models.load_model(disc_path)
    print(f'✓ Discriminator loaded from: {disc_path}')

    # Load tokenizer
    tokenizer_path = os.path.join(save_dir, 'tokenizer.pickle')
    with open(tokenizer_path, 'rb') as f:
        loaded_tokenizer = pickle.load(f)
    print(f'✓ Tokenizer loaded from: {tokenizer_path}')

    # Load metadata
    metadata_path = os.path.join(save_dir, 'metadata.json')
    with open(metadata_path, 'r') as f:
        loaded_metadata = json.load(f)
    print(f'✓ Metadata loaded from: {metadata_path}')

    # Load training history
    history_path = os.path.join(save_dir, 'training_history.json')
    with open(history_path, 'r') as f:
        loaded_history = json.load(f)
    print(f'✓ Training history loaded from: {history_path}')

    print('='*80)
    print('ALL MODELS AND DATA LOADED SUCCESSFULLY!')
    print('='*80)

    return loaded_generator, loaded_discriminator, loaded_tokenizer, loaded_metadata, loaded_history

print('Save/Load functions defined')

In [ ]:
# Save models
save_models(SAVE_DIR)

print('\nModels saved! You can now load them in a new session using:')
print('generator, discriminator, tokenizer, metadata, history = load_models(SAVE_DIR)')

## Section 10.0: Future Work and API Deployment

### Potential Improvements:

**1. Model Enhancements**:
- Replace LSTM with Transformer architecture
- Implement LeakGAN Manager-Worker framework
- Add attention mechanisms
- Multi-modal conditioning (emotions, actions)
- Hierarchical generation (sentence → paragraph)

**2. Training Improvements**:
- Implement proper Monte Carlo rollouts
- Add curriculum learning
- Use WGAN-GP (Wasserstein GAN with Gradient Penalty)
- Spectral normalization for stability
- Progressive training (increase sequence length gradually)

**3. Evaluation Enhancements**:
- Human evaluation interface
- Automatic quality metrics (METEOR, ROUGE, BERTScore)
- Character voice consistency scoring
- Dialogue flow analysis

**4. Data Augmentation**:
- Back-translation
- Paraphrasing
- Character style transfer
- Multi-season data integration

### API Deployment:

#### Flask Example:

```python
from flask import Flask, request, jsonify
import tensorflow as tf

app = Flask(__name__)

# Load models
generator = tf.keras.models.load_model('path/to/generator')
tokenizer = pickle.load(open('path/to/tokenizer.pickle', 'rb'))

@app.route('/generate', methods=['POST'])
def generate_dialogue():
    data = request.json
    seed = data.get('seed', '')
    character = data.get('character', 'Eren')
    scene = data.get('scene', 'Battle')
    temperature = data.get('temperature', 0.8)
    
    # Generate dialogue
    result = autocomplete_dialogue(
        seed, character, scene,
        temperature=temperature
    )
    
    return jsonify({
        'dialogue': result[0][0],
        'quality_score': result[0][1]
    })

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
```

#### FastAPI Example:

```python
from fastapi import FastAPI
from pydantic import BaseModel
import tensorflow as tf

app = FastAPI()

class GenerateRequest(BaseModel):
    seed: str = ''
    character: str = 'Eren'
    scene: str = 'Battle'
    temperature: float = 0.8
    num_completions: int = 3

@app.post('/generate')
async def generate_dialogue(req: GenerateRequest):
    completions = autocomplete_dialogue(
        req.seed, req.character, req.scene,
        num_completions=req.num_completions,
        temperature=req.temperature
    )
    
    return {
        'completions': [
            {'text': text, 'quality': score}
            for text, score in completions
        ]
    }
```

### Deployment Considerations:

1. **Model Serving**: TensorFlow Serving, TorchServe
2. **Containerization**: Docker for consistent deployment
3. **Scaling**: Kubernetes for auto-scaling
4. **Monitoring**: Track latency, throughput, errors
5. **Caching**: Cache common requests
6. **Rate Limiting**: Prevent abuse
7. **Authentication**: API keys or OAuth

### Production Checklist:

- [ ] Model optimization (quantization, pruning)
- [ ] Batch inference for efficiency
- [ ] Error handling and fallbacks
- [ ] Logging and monitoring
- [ ] Load testing
- [ ] Security audit
- [ ] Documentation (API docs, examples)
- [ ] CI/CD pipeline

### Usage Examples:

```bash
# cURL example
curl -X POST http://localhost:5000/generate \
  -H "Content-Type: application/json" \
  -d '{
    "seed": "I will destroy",
    "character": "Eren",
    "scene": "Battle",
    "temperature": 0.8
  }'
```

```python
# Python client example
import requests

response = requests.post(
    'http://localhost:5000/generate',
    json={
        'seed': 'We need to',
        'character': 'Armin',
        'scene': 'Strategy Meeting',
        'temperature': 0.7
    }
)

result = response.json()
print(result['dialogue'])
```

In [ ]:
# Final summary
print('='*80)
print('NOTEBOOK COMPLETE!')
print('='*80)
print('\nThis notebook implemented:')
print('✓ SeqGAN/LeakGAN concepts and theory')
print('✓ Data preparation and preprocessing')
print('✓ Generator (LSTM-based) architecture')
print('✓ Discriminator (CNN-based) architecture')
print('✓ Policy gradient (REINFORCE) implementation')
print('✓ Complete adversarial training loop')
print('✓ Interactive autocomplete tool')
print('✓ Comprehensive evaluation metrics')
print('✓ Character and scene conditioning')
print('✓ Advanced visualizations')
print('✓ Model persistence (save/load)')
print('✓ API deployment examples')
print('\n' + '='*80)
print('Thank you for using this notebook!')
print('For questions or issues, contact: support@deeplearningacademy.io')
print('='*80)